## Examples of types of interactions
In this note book we provide some examples of how to define local, nonlocal, single and multi-channel interactions.

In [1]:
import numpy as np

from jitr import reactions, rmatrix
from jitr.reactions.potentials import coulomb_charged_sphere, woods_saxon_potential
from jitr.utils import delta, kinematics, mass, constants

In [2]:
solver = rmatrix.Solver(nbasis=40)

In [3]:
# a local scalar interaction
def interaction_local(r, V0, W0, R0, a0, Zz):
    nuclear = woods_saxon_potential(r, V0, W0, R0, a0)
    coulomb = coulomb_charged_sphere(r, Zz, R0)
    return nuclear + coulomb

In [4]:
# a nonlocal scalar interaction
# note this exp(-(r-rp)**2) form is just made up for example purposes
# this is not the Perey-Buck form, which is actually l-dependent
def interaction_nonlocal(r, rp, V0, W0, R0, a0, Zz):
    nuclear = woods_saxon_potential(0.5 * (r + rp), V0, W0, R0, a0) * np.exp(
        (r - rp) ** 2
    )
    return nuclear

In [5]:
def interaction_2level(r, V0, W0, V1, W1, g, R0, a0, Zz):
    nuclear0 = woods_saxon_potential(r, V0, W0, R0, a0)
    nuclear1 = woods_saxon_potential(r, V1, W1, R0, a0)
    coupling = woods_saxon_potential(r, g * V0, 0, R0, a0)
    coulomb = coulomb_charged_sphere(r, Zz, R0)
    return np.array([[nuclear0 + coulomb, coupling], [coupling, nuclear1 + coulomb]])

In [6]:
def interaction_2level_nonlocal(r, rp, V0, W0, V1, W1, g, R0, a0, Zz):
    nuclear0 = woods_saxon_potential(0.5 * (r + rp), V0, W0, R0, a0) * np.exp(
        -((r - rp) ** 2)
    )
    nuclear1 = woods_saxon_potential(0.5 * (r + rp), V1, W1, R0, a0) * np.exp(
        -((r - rp) ** 2)
    )
    coupling = woods_saxon_potential(0.5 * (r + rp), g * V0, 0, R0, a0) * np.exp(
        -((r - rp) ** 2)
    )
    return np.array([[nuclear0, coupling], [coupling, nuclear1]])

In [7]:
# coupling between levels  within a partial wave
def coupling_2level(l):
    return np.array([[1, 1 / np.sqrt(2)], [1 / np.sqrt(2), 1]])

### Define the scattering system

In [15]:
Elab = 35  # MeV
Ca48 = (48, 20)
proton = (1, 1)

mass_target = mass.mass(*Ca48)[0]
mass_projectile = constants.MASS_P
Zz = Ca48[1] * proton[1]

In [16]:
Ecm, mu, k, eta = kinematics.classical_kinematics(
    mass_target,
    mass_projectile,
    Elab,
    Zz,
)

In [18]:
# first the 1 level system
sys_1level = reactions.ProjectileTargetSystem(
    channel_radius=5 * np.pi,
    lmax=30,
    mass_target=mass_target,
    mass_projectile=mass_projectile,
    Ztarget=Ca48[1],
    Zproj=proton[1],
)
channels_1, asymptotics_1 = sys_1level.get_partial_wave_channels(Ecm, mu, k, eta)

In [19]:
# now the 2-level system
sys_2level = reactions.ProjectileTargetSystem(
    channel_radius=5 * np.pi,
    lmax=30,
    mass_target=mass_target,
    mass_projectile=mass_projectile,
    Ztarget=Ca48[1],
    Zproj=proton[1],
    coupling=coupling_2level,
)
channels_2, asymptotics_2 = sys_2level.get_partial_wave_channels(Ecm, mu, k, eta)

In [20]:
channels_2[0].couplings

array([[1.        , 0.70710678],
       [0.70710678, 1.        ]])

In [21]:
# grab some parameters out of a hat
V0 = 42.0
W0 = 18.1
R0 = 4.8
a0 = 0.7
V1 = V0
W1 = W0
g = 1 / np.sqrt(2)

# the single channel interactions are equivalent other than the non-locality
params_local = (V0, W0, R0, a0, Zz)
params_nonlocal = params_local

# the 2-level interactions  are likewise equivalent other than the non-locality
params_2level = (V0, W0, V1, W1, g, R0, a0, Zz)
params_2level_nonlocal = params_2level

# the non-local potentials also have a local Coulomb term
params_coul = (Zz, R0)

### Let's compare the 1-level systems

In [22]:
l = 0

In [23]:
R, S, uext_boundary = solver.solve(
    channels_1[l],
    asymptotics_1[l],
    local_interaction=interaction_local,
    local_args=params_local,
)
phase_shift, phase_attenuation = delta(S[0, 0])
print(f"phase shift: {phase_shift:1.3f} + i ({phase_attenuation:1.3f}) [degrees]")

phase shift: -60.350 + i (-33.824) [degrees]


In [24]:
R, S, uext_boundary = solver.solve(
    channels_1[l],
    asymptotics_1[l],
    local_interaction=coulomb_charged_sphere,
    local_args=params_coul,
    nonlocal_interaction=interaction_nonlocal,
    nonlocal_args=params_nonlocal,
)
phase_shift, phase_attenuation = delta(S[0, 0])
print(f"phase shift: {phase_shift:1.3f} + i ({phase_attenuation:1.3f}) [degrees]")

phase shift: -60.015 + i (-0.073) [degrees]


### Let's compare the 2-level systems


In [25]:
R, S, uext_boundary = solver.solve(
    channels_2[l],
    asymptotics_2[l],
    local_interaction=interaction_2level,
    local_args=params_2level,
)
for i in range(2):
    for j in range(2):
        phase_shift, phase_attenuation = delta(S[i, j])
        print(
            f"({i},{j}): phase shift: {phase_shift} + i ({phase_attenuation}) [degrees]"
        )

(0,0): phase shift: -33.13540247803556 + i (-69.96409622470767) [degrees]
(0,1): phase shift: 60.478378904866474 + i (-72.20703247880596) [degrees]
(1,0): phase shift: 60.478378904867796 + i (-72.20703247880425) [degrees]
(1,1): phase shift: -33.135402478044554 + i (-69.96409622471776) [degrees]


In [26]:
R, S, uext_boundary = solver.solve(
    channels_2[l],
    asymptotics_2[l],
    nonlocal_interaction=interaction_2level_nonlocal,
    nonlocal_args=params_2level_nonlocal,
)
for i in range(2):
    for j in range(2):
        phase_shift, phase_attenuation = delta(S[i, j])
        print(
            f"({i},{j}): phase shift: {phase_shift} + i ({phase_attenuation}) [degrees]"
        )

(0,0): phase shift: -48.027492813205264 + i (-80.98876851969648) [degrees]
(0,1): phase shift: 44.2570945589296 + i (-78.80815621264539) [degrees]
(1,0): phase shift: 44.25709455892857 + i (-78.80815621264443) [degrees]
(1,1): phase shift: -48.02749281320191 + i (-80.98876851968703) [degrees]


## Let's compare speed

In [27]:
%%timeit
R, S, uext_boundary = solver.solve(
    channels_1[l],
    asymptotics_1[l],
    local_interaction=interaction_local,
    local_args=params_local,
)

2.39 ms ± 85.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
%%timeit
R, S, uext_boundary = solver.solve(
    channels_1[l],
    asymptotics_1[l],
    local_interaction=coulomb_charged_sphere,
    local_args=params_coul,
    nonlocal_interaction=interaction_nonlocal,
    nonlocal_args=params_nonlocal,
)

2.52 ms ± 110 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [29]:
%%timeit
R, S, uext_boundary = solver.solve(
    channels_2[l],
    asymptotics_2[l],
    local_interaction=interaction_2level,
    local_args=params_2level,
)

5.54 ms ± 337 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [30]:
%%timeit
R, S, uext_boundary = solver.solve(
    channels_2[l],
    asymptotics_2[l],
    nonlocal_interaction=interaction_2level_nonlocal,
    nonlocal_args=params_2level_nonlocal,
)

5.51 ms ± 297 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
